In [1]:
import os
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

# Define paths
ORIGINAL_DATASET_DIR = 'Refined_Hybrid_Dataset SHA-256/train'
AUGMENTED_DATASET_DIR = 'Augmented_Refined_Hybrid_Dataset/train'

# Target count for all classes
TARGET_COUNT = 5599

# Augmentation settings
augmentor = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Create new dataset directory
if os.path.exists(AUGMENTED_DATASET_DIR):
    shutil.rmtree(AUGMENTED_DATASET_DIR)  # Clear old data
os.makedirs(AUGMENTED_DATASET_DIR, exist_ok=True)

# Loop through each class
for class_name in os.listdir(ORIGINAL_DATASET_DIR):
    original_class_path = os.path.join(ORIGINAL_DATASET_DIR, class_name)
    augmented_class_path = os.path.join(AUGMENTED_DATASET_DIR, class_name)

    os.makedirs(augmented_class_path, exist_ok=True)

    image_filenames = os.listdir(original_class_path)
    num_original_images = len(image_filenames)

    # Skip augmentation for "happy"
    if class_name == "happy":
        for img_file in image_filenames:
            shutil.copy(os.path.join(original_class_path, img_file), augmented_class_path)
        print(f"✅ Skipped augmentation for {class_name} (already at {TARGET_COUNT})")
        continue

    # Calculate number of images needed
    num_needed = TARGET_COUNT - num_original_images
    print(f"🔹 {class_name}: {num_original_images} images, need {num_needed} more")

    # Copy original images first
    for img_file in image_filenames:
        shutil.copy(os.path.join(original_class_path, img_file), augmented_class_path)

    # Augment images to reach the target count
    i = 0
    while num_needed > 0:
        img_file = image_filenames[i % num_original_images]  # Loop over original images
        img_path = os.path.join(original_class_path, img_file)

        img = load_img(img_path, color_mode="grayscale", target_size=(48, 48))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        aug_iter = augmentor.flow(img_array, batch_size=1)
        aug_img = next(aug_iter)[0].astype(np.uint8)
        aug_img = array_to_img(aug_img)

        aug_img.save(os.path.join(augmented_class_path, f"aug_{i}_{img_file}"))
        num_needed -= 1
        i += 1

print("✅ Augmentation completed! All classes now have 5599 images.")


🔹 angry: 2970 images, need 2629 more
🔹 disgust: 4539 images, need 1060 more
🔹 fear: 3015 images, need 2584 more
✅ Skipped augmentation for happy (already at 5599)
🔹 neutral: 3931 images, need 1668 more
🔹 sad: 3702 images, need 1897 more
🔹 surprise: 3082 images, need 2517 more
✅ Augmentation completed! All classes now have 5599 images.
